In [7]:
import numpy as np
import itertools
from sage.symbolic.expression_conversions import polynomial

def calculate_sigma_mgf_explicitly_mod_degree(n, l):
    # # We want to look at all terms of degree at most n
    # for degree in range(n):

    # Find the permutations
    symmetric_group_lists_of_eigenvalues = []
    for permutation in SymmetricGroup(n):
            # A cycle of size m has eigenvalues that are m-roots of unity.
            # e.g. if zeta is an m-root of unity, its corresponding eigenvector is (zeta, zeta^2, ..., zeta^m)
            eigenvalues = []
            for cycle_len in permutation.cycle_type():
                # The cycle type is a collection of numbers corresponding to sizes of each disjoint cycle
                for i in range(cycle_len):
                    eigenvalues.append(
                        e^(2 * I * pi * i / cycle_len)
                    )
            assert len(eigenvalues) == n
            symmetric_group_lists_of_eigenvalues.append(eigenvalues)

    # Convert the permutations into generalised permutations
    multiplied_eigenvalues = []
    for eigenvalues in symmetric_group_lists_of_eigenvalues:
        # Find all the coefficients corresponding to the non-0 entries of the generalised permutation matrix
        # TODO: there is probably a way to exploit symmetry here to make this more efficient
        for indices_of_coefficients in itertools.product(range(l), repeat=n):
            # Then each of the eigenvalues of the symmetric group gets multiplied by these element-wise
            multiplied_eigenvalues.append([e^(2 * I * pi * index_of_coef / l) * eigenval for (index_of_coef, eigenval) in zip(indices_of_coefficients, eigenvalues)])

    
    Sym = SymmetricFunctions(CC)
    m = Sym.m()
    h = Sym.h()
    total = 0
    # Get the average for each partition, up to and including degree n
    for degree in range(n+1):
        for tau in Partitions(degree):
            m_tau = m(tau)
            h_tau_poly = h(tau).expand(n)
            total_per_tau = 0
            for multiplied_eigenvalue in multiplied_eigenvalues:
                if n == 2 and l == 2:
                    print(tau, h_tau_poly(multiplied_eigenvalue))
                total_per_tau += h_tau_poly(multiplied_eigenvalue)
            avg_per_tau = total_per_tau / len(multiplied_eigenvalues)
            total += avg_per_tau * m_tau
    return total

In [8]:
# This extends the idea behind the truncate function on MPolynomial, but now truncating based on total degree
def truncate(poly, n):
    R = poly.parent()
    
    return R({key: coef for key, coef in poly.monomial_coefficients().items()
              if sum(key) <= n})

In [9]:
def calculate_sigma_mgf_with_formula_mod_degree(n, l):
    
    # Polynomial ring in n polys
    R = PolynomialRing(CC, n, "x")
    variables = R.gens()

    product = R(1)
    for degree in range(l, n+1, l):
        # TODO: there is probably a more effecient way to do this
        # Consider all possible monomials of this degree
        seen_monomials = set()
        for selected_vars in itertools.combinations_with_replacement(variables, degree):
            monomial = 1
            for var in selected_vars:
                monomial *= var
            if monomial in seen_monomials:
                continue
            # Now create a truncated version of 1/(1-monomial)=1 + monomial + monomial ^ 2 + ...
            monomial_power = 1
            total = 0
            for _ in range(0, n+1, degree):
                total += monomial_power
                monomial_power *= monomial
            product *= total
            seen_monomials.add(monomial)
            product = truncate(product, n)
    
    return product

In [10]:
def symmetric_poly_to_poly(poly, n):
    poly = poly.expand(n)
    # There is weird behaviour in Sage where this returns a polynomial with just one term, which itself is a symbolic term containing the whole thing
    poly = list(poly.monomial_coefficients().values())
    assert len(poly) == 1
    return polynomial(poly[0], CC)

In [11]:
n = 3
l = 1

explicit_result = symmetric_poly_to_poly(calculate_sigma_mgf_explicitly_mod_degree(n, l), n)
print(explicit_result)

formula_result = calculate_sigma_mgf_with_formula_mod_degree(n, l)
print(formula_result)

(3.00000000000000 - 1.85037170770859e-17*I)*x0^3 + (4.00000000000000 + 3.70074341541719e-17*I)*x0^2*x1 + (4.00000000000000 + 3.70074341541719e-17*I)*x0*x1^2 + (3.00000000000000 - 1.85037170770859e-17*I)*x1^3 + (4.00000000000000 + 3.70074341541719e-17*I)*x0^2*x2 + (5.00000000000000 + 3.70074341541719e-17*I)*x0*x1*x2 + (4.00000000000000 + 3.70074341541719e-17*I)*x1^2*x2 + (4.00000000000000 + 3.70074341541719e-17*I)*x0*x2^2 + (4.00000000000000 + 3.70074341541719e-17*I)*x1*x2^2 + (3.00000000000000 - 1.85037170770859e-17*I)*x2^3 + 2.00000000000000*x0^2 + 2.00000000000000*x0*x1 + 2.00000000000000*x1^2 + 2.00000000000000*x0*x2 + 2.00000000000000*x1*x2 + 2.00000000000000*x2^2 + x0 + x1 + x2 + 1.00000000000000
3.00000000000000*x0^3 + 4.00000000000000*x0^2*x1 + 4.00000000000000*x0*x1^2 + 3.00000000000000*x1^3 + 4.00000000000000*x0^2*x2 + 5.00000000000000*x0*x1*x2 + 4.00000000000000*x1^2*x2 + 4.00000000000000*x0*x2^2 + 4.00000000000000*x1*x2^2 + 3.00000000000000*x2^3 + 2.00000000000000*x0^2 + 2.0

In [12]:
for n in range(1, 5):
    for l in range(1, 5):
        print(f"n: {n}, l: {l}")
        explicit_result = symmetric_poly_to_poly(calculate_sigma_mgf_explicitly_mod_degree(n, l), n)        
        formula_result = truncate(calculate_sigma_mgf_with_formula_mod_degree(n, l), n)

        for coef_diff in (explicit_result - formula_result).monomial_coefficients().values():
            assert abs(coef_diff) < 0.000001, f"Difference of coefficient is {coef_diff} for explicit result {explicit_result} and predicted result {formula_result}"

n: 1, l: 1
n: 1, l: 2
n: 1, l: 3
n: 1, l: 4
n: 2, l: 1
n: 2, l: 2
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[] 1.00000000000000
[1] 2.00000000000000
[1] 0.000000000000000
[1] 0.000000000000000
[1] -2.00000000000000
[1] 0.000000000000000
[1] 2.00000000000000
[1] -2.00000000000000
[1] 0.000000000000000
[2] 3.00000000000000
[2] 1.00000000000000
[2] 1.00000000000000
[2] 3.00000000000000
[2] 1.00000000000000
[2] 3.00000000000000
[2] 3.00000000000000
[2] 1.00000000000000
[1, 1] 4.00000000000000
[1, 1] 0.000000000000000
[1, 1] 0.000000000000000
[1, 1] 4.00000000000000
[1, 1] 0.000000000000000
[1, 1] 4.00000000000000
[1, 1] 4.00000000000000
[1, 1] 0.000000000000000


AssertionError: Difference of coefficient is 1.00000000000000 for explicit result 2.00000000000000*x0^2 + 2.00000000000000*x0*x1 + 2.00000000000000*x1^2 + 1.00000000000000 and predicted result x0^2 + x0*x1 + x1^2 + 1.00000000000000